In [29]:
import os
import json
import requests
from pycocotools.coco import COCO
import numpy as np
import pandas as pd

In [33]:
annotation_file = '/Users/psjj/Downloads/coco2017/annotations/instances_train2017.json'
coco = COCO(annotation_file)
BASE_PATH = '/Users/psjj/Downloads/coco2017/'

loading annotations into memory...
Done (t=9.66s)
creating index...
index created!


In [27]:
with open(f'{BASE_PATH}/annotations/captions_train2017.json', 'r') as f:
    data = json.load(f)
    data =  data['annotations']     

In [30]:
img_cap_pairs = []

for sample in data:
    img_name = '%012d.jpg' % sample['image_id']
    img_cap_pairs.append([img_name, sample['caption']])

captions = pd.DataFrame(img_cap_pairs, columns=['image', 'caption'])
captions['image'] = captions['image'].apply(
    lambda x: f'{BASE_PATH}/train2017/{x}'
)
captions = captions.sample(70000)
captions = captions.reset_index(drop=True)
captions.head()

,image,caption
0,/Users/psjj/Downloads/coco2017//train2017/0000...,A little baby sitting next to a small dog and ...
1,/Users/psjj/Downloads/coco2017//train2017/0000...,A bowl sink on a granite slab in a bathroom
2,/Users/psjj/Downloads/coco2017//train2017/0000...,A woman and some kids sitting on a couch.
3,/Users/psjj/Downloads/coco2017//train2017/0000...,Well decorated restroom with sink and chair fo...
4,/Users/psjj/Downloads/coco2017//train2017/0000...,Basket of stuffed animals next to a cake decor...


In [35]:
categories = coco.loadCats(coco.getCatIds())
categories

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

In [36]:
for category in categories:
    print(f"ID: {category['id']}, 클래스 이름: {category['name']}, 상위 카테고리: {category['supercategory']}")

ID: 1, 클래스 이름: person, 상위 카테고리: person
ID: 2, 클래스 이름: bicycle, 상위 카테고리: vehicle
ID: 3, 클래스 이름: car, 상위 카테고리: vehicle
ID: 4, 클래스 이름: motorcycle, 상위 카테고리: vehicle
ID: 5, 클래스 이름: airplane, 상위 카테고리: vehicle
ID: 6, 클래스 이름: bus, 상위 카테고리: vehicle
ID: 7, 클래스 이름: train, 상위 카테고리: vehicle
ID: 8, 클래스 이름: truck, 상위 카테고리: vehicle
ID: 9, 클래스 이름: boat, 상위 카테고리: vehicle
ID: 10, 클래스 이름: traffic light, 상위 카테고리: outdoor
ID: 11, 클래스 이름: fire hydrant, 상위 카테고리: outdoor
ID: 13, 클래스 이름: stop sign, 상위 카테고리: outdoor
ID: 14, 클래스 이름: parking meter, 상위 카테고리: outdoor
ID: 15, 클래스 이름: bench, 상위 카테고리: outdoor
ID: 16, 클래스 이름: bird, 상위 카테고리: animal
ID: 17, 클래스 이름: cat, 상위 카테고리: animal
ID: 18, 클래스 이름: dog, 상위 카테고리: animal
ID: 19, 클래스 이름: horse, 상위 카테고리: animal
ID: 20, 클래스 이름: sheep, 상위 카테고리: animal
ID: 21, 클래스 이름: cow, 상위 카테고리: animal
ID: 22, 클래스 이름: elephant, 상위 카테고리: animal
ID: 23, 클래스 이름: bear, 상위 카테고리: animal
ID: 24, 클래스 이름: zebra, 상위 카테고리: animal
ID: 25, 클래스 이름: giraffe, 상위 카테고리: animal
ID: 27, 클래스 이름: backpack, 상위 카

In [16]:
filter_classes = [
    'car', 'bus', 'truck', 'motorcycle', 'bicycle', 'train', 'boat', 'airplane',
    'traffic light', 'stop sign', 'fire hydrant', 'parking meter', 'bench',
     'person', 'dog', 'cat', 'bird', 'horse',
                  'sheep', 'cow', 'zebra',
    'backpack', 'handbag', 'umbrella', 'suitcase', 'chair', 'potted plant',
    'bench', 'bottle', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'pizza', 'donut', 
    'cake', 'cell phone', 'laptop', 'tv', 'mouse', 'remote', 'keyboard'
]


18


In [38]:
import os
from pycocotools.coco import COCO

# COCO 어노테이션 파일 경로
annotation_file = '/Users/psjj/Downloads/coco2017/annotations/instances_train2017.json'
coco = COCO(annotation_file)

# 필터링할 클래스의 카테고리 ID 목록
filter_class_ids = [3, 6, 8, 4, 2, 7, 9, 5, 10, 13, 11, 14, 15, 1, 18, 17, 16, 19]

# 해당 카테고리를 포함한 이미지 ID 가져오기
img_ids = coco.getImgIds(catIds=filter_class_ids)
print(f"필터링된 이미지 ID 개수: {len(img_ids)}")

loading annotations into memory...
Done (t=11.47s)
creating index...
index created!
필터링된 이미지 ID 개수: 0


In [21]:
cat_ids = coco.getCatIds(catNms=filter_classes)

In [39]:
import json

# COCO JSON 파일 경로
json_file_path = '/Users/psjj/Downloads/coco2017/annotations/instances_train2017.json'

# JSON 파일 열기
with open(json_file_path, 'r') as f:
    coco_data = json.load(f)

# 'categories' 섹션 확인 (클래스 정보)
categories = coco_data['categories']
print("COCO Categories:")
for category in categories:
    print(f"ID: {category['id']}, Name: {category['name']}")

# 'images' 섹션 확인 (첫 번째 이미지 정보 확인)
images = coco_data['images']
print("\nFirst image data:")
print(images[0])

# 'annotations' 섹션 확인 (첫 번째 어노테이션 정보 확인)
annotations = coco_data['annotations']
print("\nFirst annotation data:")
print(annotations[0])


COCO Categories:
ID: 1, Name: person
ID: 2, Name: bicycle
ID: 3, Name: car
ID: 4, Name: motorcycle
ID: 5, Name: airplane
ID: 6, Name: bus
ID: 7, Name: train
ID: 8, Name: truck
ID: 9, Name: boat
ID: 10, Name: traffic light
ID: 11, Name: fire hydrant
ID: 13, Name: stop sign
ID: 14, Name: parking meter
ID: 15, Name: bench
ID: 16, Name: bird
ID: 17, Name: cat
ID: 18, Name: dog
ID: 19, Name: horse
ID: 20, Name: sheep
ID: 21, Name: cow
ID: 22, Name: elephant
ID: 23, Name: bear
ID: 24, Name: zebra
ID: 25, Name: giraffe
ID: 27, Name: backpack
ID: 28, Name: umbrella
ID: 31, Name: handbag
ID: 32, Name: tie
ID: 33, Name: suitcase
ID: 34, Name: frisbee
ID: 35, Name: skis
ID: 36, Name: snowboard
ID: 37, Name: sports ball
ID: 38, Name: kite
ID: 39, Name: baseball bat
ID: 40, Name: baseball glove
ID: 41, Name: skateboard
ID: 42, Name: surfboard
ID: 43, Name: tennis racket
ID: 44, Name: bottle
ID: 46, Name: wine glass
ID: 47, Name: cup
ID: 48, Name: fork
ID: 49, Name: knife
ID: 50, Name: spoon
ID: 51,

In [41]:
import json

# COCO 캡션 어노테이션 파일 경로
caption_file_path = '/Users/psjj/Downloads/coco2017/annotations/captions_train2017.json'

# JSON 파일 열기
with open(caption_file_path, 'r') as f:
    captions_data = json.load(f)

# 키워드 기반으로 필터링할 키워드 리스트
filter_keywords = ['car', 'bus', 'truck', 'motorcycle', 'bicycle', 'train', 'boat', 'airplane',
    'traffic light', 'stop sign', 'fire hydrant', 'parking meter', 'bench',
     'person', 'dog', 'cat', 'bird', 'horse',
                  'sheep', 'cow', 'zebra','laptop', 'cell phone',
    'backpack', 'handbag', 'umbrella', 'suitcase', 'chair', 'potted plant',
    'bench', 'bottle', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'pizza', 'donut', 
    'cake', 'cell phone', 'laptop', 'tv', 'mouse', 'remote', 'keyboard']
 # 예시로 사용될 키워드

# 필터링된 이미지 ID 리스트를 저장할 리스트
filtered_img_ids = set()

# 어노테이션 섹션을 순회하면서 캡션을 필터링
for annotation in captions_data['annotations']:
    caption = annotation['caption'].lower()  # 캡션을 소문자로 변환해 비교
    if any(keyword in caption for keyword in filter_keywords):
        filtered_img_ids.add(annotation['image_id'])

print(f"필터링된 이미지 개수: {len(filtered_img_ids)}")

# 필터링된 이미지 ID 리스트 출력 (일부만 출력)
print(f"첫 10개의 필터링된 이미지 ID: {list(filtered_img_ids)[:10]}")

필터링된 이미지 개수: 65340
첫 10개의 필터링된 이미지 ID: [262145, 262146, 524291, 262148, 131075, 393224, 393225, 131084, 393230, 131087]


## 데이터 분포 체크

In [45]:
import json
from collections import defaultdict

# COCO 어노테이션 파일 경로
instances_file = '/Users/psjj/Downloads/coco2017/annotations/instances_train2017.json'
captions_file = '/Users/psjj/Downloads/coco2017/annotations/captions_train2017.json'

# JSON 파일 열기
with open(instances_file, 'r') as f:
    instances_data = json.load(f)

with open(captions_file, 'r') as f:
    captions_data = json.load(f)

# 카테고리 ID와 이름 매핑
category_mapping = {category['id']: category['name'] for category in instances_data['categories']}

# 클래스별 캡션 데이터 분포를 저장할 딕셔너리
class_caption_count = defaultdict(int)

# 이미지 ID와 해당 클래스 ID 매핑
image_class_mapping = defaultdict(list)
for ann in instances_data['annotations']:
    image_class_mapping[ann['image_id']].append(ann['category_id'])

# 캡션 데이터를 순회하며 해당 이미지가 어떤 클래스를 포함하는지 확인하고 카운팅
for annotation in captions_data['annotations']:
    image_id = annotation['image_id']
    caption = annotation['caption']
    
    if image_id in image_class_mapping:
        class_ids = image_class_mapping[image_id]
        for class_id in class_ids:
            class_name = category_mapping[class_id]
            class_caption_count[class_name] += 1

sorted_class_caption_count = sorted(class_caption_count.items(), key=lambda x: x[1], reverse=True)

# 정렬된 결과 출력
for class_name, count in sorted_class_caption_count:
    print(f"Class '{class_name}': {count} captions")


Class 'person': 1312984 captions
Class 'car': 219416 captions
Class 'chair': 192563 captions
Class 'book': 123644 captions
Class 'bottle': 121816 captions
Class 'cup': 103283 captions
Class 'dining table': 78622 captions
Class 'bowl': 71829 captions
Class 'traffic light': 64474 captions
Class 'handbag': 61785 captions
Class 'umbrella': 57180 captions
Class 'bird': 54077 captions
Class 'boat': 53826 captions
Class 'truck': 49885 captions
Class 'bench': 49224 captions
Class 'sheep': 47566 captions
Class 'banana': 47333 captions
Class 'kite': 45410 captions
Class 'motorcycle': 43655 captions
Class 'backpack': 43618 captions
Class 'potted plant': 43270 captions
Class 'cow': 40775 captions
Class 'wine glass': 39579 captions
Class 'carrot': 39281 captions
Class 'knife': 38881 captions
Class 'broccoli': 36565 captions
Class 'donut': 35927 captions
Class 'bicycle': 35584 captions
Class 'skis': 33245 captions
Class 'vase': 33086 captions
Class 'horse': 32937 captions
Class 'tie': 32493 captions

In [49]:
import json
import os
import requests

# COCO 캡션 어노테이션 파일 경로
caption_file_path = '/Users/psjj/Downloads/coco2017/annotations/captions_train2017.json'
instances_file_path = '/Users/psjj/Downloads/coco2017/annotations/instances_train2017.json'

# JSON 파일 열기
with open(caption_file_path, 'r') as f:
    captions_data = json.load(f)

with open(instances_file_path, 'r') as f:
    instances_data = json.load(f)

# 키워드 기반으로 필터링할 키워드 리스트
filter_keywords = ['car', 'bus', 'truck', 'motorcycle', 'bicycle', 'train', 'boat', 'airplane',
    'traffic light', 'stop sign', 'fire hydrant', 'parking meter', 'bench',
     'person', 'dog', 'cat', 'bird', 'horse', 'sheep', 'cow', 'zebra', 
     'backpack', 'handbag', 'umbrella', 'suitcase', 'chair', 'potted plant',
     'bottle', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 
     'sandwich', 'orange', 'broccoli', 'carrot', 'pizza', 'donut', 'cake', 
     'cell phone', 'laptop', 'tv', 'mouse', 'remote', 'keyboard']

# 필터링된 이미지 ID 리스트를 저장할 리스트
filtered_img_ids = set()

# 어노테이션 섹션을 순회하면서 캡션을 필터링
for annotation in captions_data['annotations']:
    caption = annotation['caption'].lower()  # 캡션을 소문자로 변환해 비교
    if any(keyword in caption for keyword in filter_keywords):
        filtered_img_ids.add(annotation['image_id'])

print(f"필터링된 이미지 개수: {len(filtered_img_ids)}")

# 이미지 저장 경로 설정
save_dir = '/Users/psjj/NIPA/filtered_dir'  # 저장할 디렉토리 경로 설정
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 이미지 정보에서 이미지 URL 가져오기
image_id_to_url = {image['id']: image['coco_url'] for image in instances_data['images']}

# 필터링된 이미지 다운로드 및 저장
for img_id in list(filtered_img_ids)[:10]:  # 첫 10개의 이미지만 다운로드 (필요에 따라 수정)
    if img_id in image_id_to_url:
        img_url = image_id_to_url[img_id]
        img_data = requests.get(img_url).content
        img_filename = os.path.join(save_dir, f"{img_id}.jpg")
        with open(img_filename, 'wb') as handler:
            handler.write(img_data)
        print(f"{img_filename} 저장 완료")


필터링된 이미지 개수: 83983
/Users/psjj/NIPA/CLIP/image_dir/262145.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/262146.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/524291.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/262148.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/9.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/262161.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/262162.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/524311.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/524314.jpg 저장 완료
/Users/psjj/NIPA/CLIP/image_dir/262171.jpg 저장 완료


In [51]:
image_id_to_url = {image['id']: image['coco_url'] for image in instances_data['images']}
image_id_to_url

{391895: 'http://images.cocodataset.org/train2017/000000391895.jpg',
 522418: 'http://images.cocodataset.org/train2017/000000522418.jpg',
 184613: 'http://images.cocodataset.org/train2017/000000184613.jpg',
 318219: 'http://images.cocodataset.org/train2017/000000318219.jpg',
 554625: 'http://images.cocodataset.org/train2017/000000554625.jpg',
 574769: 'http://images.cocodataset.org/train2017/000000574769.jpg',
 60623: 'http://images.cocodataset.org/train2017/000000060623.jpg',
 309022: 'http://images.cocodataset.org/train2017/000000309022.jpg',
 5802: 'http://images.cocodataset.org/train2017/000000005802.jpg',
 222564: 'http://images.cocodataset.org/train2017/000000222564.jpg',
 118113: 'http://images.cocodataset.org/train2017/000000118113.jpg',
 193271: 'http://images.cocodataset.org/train2017/000000193271.jpg',
 224736: 'http://images.cocodataset.org/train2017/000000224736.jpg',
 483108: 'http://images.cocodataset.org/train2017/000000483108.jpg',
 403013: 'http://images.cocodataset.o